In [1]:
using HTTP
using Dates

In [2]:
#=
    DarkSky API has 2 features: Forecast and Timemachine.  Timemachine allows you to obtain historical
    weather data.
    format
        forecast_url = https://api.darksky.net/forecast/[key]/[latitude],[longitude]

        timemachine_url = https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]
            time is unix timestamp. 

=#
 #=
    For historical data, the unix_timestamp would have to be an array spaning the time frame of the desired
    historical data.

    Data has to be saved into a CSV format
=#
#= 
    In order to work with the earthquake dates which are given in UTC in the following form
    2018-12-29T03:39:09.740Z
    The "Z" indicating Zulu? or UTC time has to be stripped from the string.  A complex example would be as follows:
    round(Int, Dates.datetime2unix(DateTime("2018-12-29T03:39:09.740Z")))
    which should give the result 
    1546054750
    which roughly translates to Dec 29, 2018 at 3:39am UTC
=#
#=
    The object provided by the weather api for historical data, includes sunrise and sunset time and perhaps temperature 
    either at sunrise/sunset or an hour that is close to that time which would be called the terminator. Time.  The first
    call to the api would give us the sunrise/sunset and that could be used to get the temperature for sunrise/sunset
    for the previous days as well as previous years
=#

In [123]:
latitude = 5.8983
longitude = 126.9209
earthquake_datetime = "2017-12-29T18:00:00.000Z"
base_url = "https://api.darksky.net/forecast"

"https://api.darksky.net/forecast"

In [124]:
function read_apikey(apikey_file)
    key = open(apikey_file) do file
          readlines(file)
        end 
    return String(key[1])
end

read_apikey (generic function with 1 method)

In [125]:
apikey = read_apikey("/home/antonio/Repos/iono2/julia_scripts/darksky_api.txt")
println("Done reading apikey!")

Done reading apikey!


In [126]:
function unix_time(earthquake_time)
    println(earthquake_time)
    # remove the last char (Z) from earthquake_time, ex, 2018-12-29T03:39:09.740Z
    eq_time = strip(earthquake_time, last(earthquake_time))
    println("Removed the last char from $earthquake_time")
    round(Int, Dates.datetime2unix(DateTime(eq_time)))
end

unix_time (generic function with 1 method)

In [127]:
function darksky_api_call(url)
    try
        response = HTTP.get(url)
        return String(response.body)
        catch e
        return "Error occured: $e"
    end
end

darksky_api_call (generic function with 1 method)

In [128]:
function historical_weather(latitude, longitude, earthquake_datetime)
    println("Looking up info for Latitude $latitude, longitud $longitude and earthquake_time $earthquake_time\n")
    unix_timestamp = unix_time(earthquake_datetime)
    historical_temperature_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitud) * "," * string(unix_timestamp)
    println("unix_time stamp : $unix_timestamp", "\n$historical_temperature_url")
    darksky_api_call(historical_temperature_url)
end

historical_weather (generic function with 1 method)

In [129]:
function forecast_weather(latitude, longitude)
    forecast_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitud)
    darksky_api_call(forecast_url)
end

forecast_weather (generic function with 1 method)

In [130]:
historical = historical_weather(latitude, longitude, earthquake_datetime)

Looking up info for Latitude 5.8983, longitud 126.9209 and earthquake_time 2018-12-29T03:39:09.740Z

2017-12-29T18:00:00.000Z
Removed the last char from 2017-12-29T18:00:00.000Z
unix_time stamp : 1514570400
https://api.darksky.net/forecast/df7160dd597d0332eb4508c91fab3853/5.8983,125.2905,1514570400


"{\"latitude\":5.8983,\"longitude\":125.2905,\"timezone\":\"Asia/Manila\",\"currently\":{\"time\":1514570400,\"uvIndex\":0},\"hourly\":{\"data\":[{\"time\":1514592000,\"summary\":\"Humid\",\"icon\":\"clear-day\",\"precipIntensity\":0,\"precipProbability\":0,\"temperature\":80.15,\"apparentTemperature\":84.54,\"dewPoint\":73.39,\"humidity\":0.8,\"windSpeed\":2.24,\"windBearing\":11,\"cloudCover\":0.19,\"uvIndex\":2,\"visibility\":6.216},{\"time\":1514595600,\"summary\":\"Humid\",\"icon\":\"clear-day\",\"precipIntensity\":0,\"precipProbability\":0,\"temperature\":83.78,\"apparentTemperature\":90.33,\"dewPoint\":73.39,\"humidity\":0.71,\"windSpeed\":6.93,\"windBearing\":20,\"cloudCover\":0.19,\"uvIndex\":4,\"visibility\":6.216},{\"time\":1514599200,\"summary\":\"Humid\",\"icon\":\"clear-day\",\"precipIntensity\":0,\"precipProbability\":0,\"temperature\":89.18,\"apparentTemperature\":99.66,\"dewPoint\":75.19,\"humidity\":0.64,\"windSpeed\":4.69,\"windBearing\":50,\"cloudCover\":0.19,\"uvIn

In [133]:
historical["Dily"]

MethodError: MethodError: no method matching getindex(::String, ::String)
Closest candidates are:
  getindex(::String, !Matched::UnitRange{Int64}) at strings/string.jl:241
  getindex(::String, !Matched::Int64) at strings/string.jl:206
  getindex(::String, !Matched::UnitRange{#s57} where #s57<:Integer) at strings/string.jl:238
  ...

In [131]:
#forecast_urls
forcast = forecast_weather(latitude, longitude)

"{\"latitude\":5.8983,\"longitude\":125.2905,\"timezone\":\"Asia/Manila\",\"currently\":{\"time\":1578109960,\"summary\":\"Humid\",\"icon\":\"clear-day\",\"precipIntensity\":0,\"precipProbability\":0,\"temperature\":85.96,\"apparentTemperature\":92.38,\"dewPoint\":72.22,\"humidity\":0.64,\"pressure\":1008.8,\"windSpeed\":8.35,\"windGust\":13.46,\"windBearing\":1,\"cloudCover\":0.01,\"uvIndex\":11,\"visibility\":10,\"ozone\":240.7},\"hourly\":{\"summary\":\"Humid and partly cloudy throughout the day.\",\"icon\":\"partly-cloudy-day\",\"data\":[{\"time\":1578106800,\"summary\":\"Humid\",\"icon\":\"clear-day\",\"precipIntensity\":0,\"precipProbability\":0,\"temperature\":84.73,\"apparentTemperature\":90.93,\"dewPoint\":72.44,\"humidity\":0.67,\"pressure\":1009.5,\"windSpeed\":7.89,\"windGust\":12.96,\"windBearing\":359,\"cloudCover\":0.01,\"uvIndex\":11,\"visibility\":10,\"ozone\":241.5},{\"time\":1578110400,\"summary\":\"Humid\",\"icon\":\"clear-day\",\"precipIntensity\":0,\"precipProbabi

In [109]:
unix_timestamp = unix_time("2018-12-29T03:39:09.740Z")

2018-12-29T03:39:09.740Z
Removed the last char from 2018-12-29T03:39:09.740Z


1546054750

In [110]:
earthquake_time

"2018-12-29T03:39:09.740Z"

In [111]:
historical_temperature_url

UndefVarError: UndefVarError: historical_temperature_url not defined